# PARAMETERS 

In [1]:
problem_name = 'roads' #to save
model_architecture = 'unet'
weights_path = None 
target_size = (256, 256) 
batch_size = 2

epochs = 100 #após x épocas sem melhorar pará (a usar callback)

In [20]:
import signal
from keras.layers import Input
from keras.optimizers import Adam, RMSprop
from keras.models import Model, Sequential
from keras.models import load_model
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.layers import Activation
from keras.constraints import maxnorm 
from keras.optimizers import SGD 
from keras.utils import np_utils 
from keras import backend as K 
from keras.utils.data_utils import get_file
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import keras
K.set_image_dim_ordering('tf') #ordem 'th' ou 'tf' 
import numpy as np
from numpy import genfromtxt
import math 

from timeit import default_timer as timer
from time import time as tick
import matplotlib.pyplot as plt 
import pickle 
from os import listdir
from PIL import Image, ImageOps
from os.path import isfile, join
import os
from scipy.misc	import toimage 
from scipy import misc, ndimage
import scipy.fftpack as pack
import scipy.misc
from scipy.ndimage import rotate
from sklearn.datasets import fetch_mldata
from sklearn.svm import SVC
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from skimage.io import imread
from skimage.transform import resize

from sklearn.metrics import roc_auc_score
import pandas as pd
import pathlib

from keras.layers import (Activation, Conv2D, Conv2DTranspose, Dense, Dropout,
                          Flatten, Input, MaxPooling2D, concatenate,
                          GlobalAveragePooling2D)

# fixar random seed para se puder reproduzir os resultados 
seed = 9 
np.random.seed(seed) 

# Útils

In [3]:
def print_model(model,fich):
    from keras.utils import plot_model
    plot_model(model, to_file=fich, show_shapes=True, show_layer_names=True)
    
def print_history_accuracy(history):
    print(history.history.keys())
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
def print_history_loss(history):
    print(history.history.keys())
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

def load_batch(fpath, label_key='labels'): 
 
    f = open(fpath, 'rb') 
    d = pickle.load(f, encoding='bytes') 
    d_decoded = {}        # decode utf8 
    for k, v in d.items(): 
        d_decoded[k.decode('utf8')] = v 
    d = d_decoded 
    f.close() 
    data = d['data'] 
    labels = d[label_key] 
    data = data.reshape(data.shape[0], 3, 32, 32) 
    return data, labels


def rotate_resize(temp, tam_image):
    #-------------------rodar se necessário e cortar em quadrado
    if temp.shape[0] > temp.shape[1]:
        temp = rotate(temp,90)
    
    #cortar em quadrado no centro da imagem e fazer resize para o tam_image
    difShapes = temp.shape[1]-temp.shape[0]
    return (255 * resize(temp[0:temp.shape[0],int(difShapes/2):int(difShapes/2)+temp.shape[0]],
                            (tam_image, tam_image))).astype(np.uint8)

# Models

In [4]:
#choosing model

def choosing_model(model_architecture, num_classes,epochs, weights_path=None):
    
    model = None
    
    if model_architecture == 'unet':
        model = unet(input_tensor, input_shape=None, nb_classes=None)
    if model == None:
        print('non valid model')
        
    return model

In [26]:
def unet(input_tensor, input_shape=None, nb_classes=None):
    # Makes 4 downsamplings
    # 2^4=16 » Minimum image size allowed is 32x32
    print("Architecture: U-Net")
    conv1 = Conv2D(32, (3, 3), activation="relu", padding="same")(input_tensor)
    conv1 = Conv2D(32, (3, 3), activation="relu", padding="same")(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation="relu", padding="same")(pool1)
    conv2 = Conv2D(64, (3, 3), activation="relu", padding="same")(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation="relu", padding="same")(pool2)
    conv3 = Conv2D(128, (3, 3), activation="relu", padding="same")(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation="relu", padding="same")(pool3)
    conv4 = Conv2D(256, (3, 3), activation="relu", padding="same")(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation="relu", padding="same")(pool4)
    conv5 = Conv2D(512, (3, 3), activation="relu", padding="same")(conv5)

    up6 = concatenate(
        [
            Conv2DTranspose(256, (2, 2), strides=(2, 2),
                            padding="same")(conv5), conv4
        ],
        axis=3)
    conv6 = Conv2D(256, (3, 3), activation="relu", padding="same")(up6)
    conv6 = Conv2D(256, (3, 3), activation="relu", padding="same")(conv6)

    up7 = concatenate(
        [
            Conv2DTranspose(128, (2, 2), strides=(2, 2),
                            padding="same")(conv6), conv3
        ],
        axis=3)
    conv7 = Conv2D(128, (3, 3), activation="relu", padding="same")(up7)
    conv7 = Conv2D(128, (3, 3), activation="relu", padding="same")(conv7)

    up8 = concatenate(
        [
            Conv2DTranspose(64, (2, 2), strides=(2, 2), padding="same")(conv7),
            conv2
        ],
        axis=3)
    conv8 = Conv2D(64, (3, 3), activation="relu", padding="same")(up8)
    conv8 = Conv2D(64, (3, 3), activation="relu", padding="same")(conv8)

    up9 = concatenate(
        [
            Conv2DTranspose(32, (2, 2), strides=(2, 2), padding="same")(conv8),
            conv1
        ],
        axis=3)
    conv9 = Conv2D(32, (3, 3), activation="relu", padding="same")(up9)
    conv9 = Conv2D(32, (3, 3), activation="relu", padding="same")(conv9)

    conv10 = Conv2D(3, (1, 1), activation="sigmoid")(conv9)

    model = Model(inputs=[input_tensor], outputs=[conv10])

    model.compile(
        optimizer=Adam(lr=1e-4),
        loss=["mean_squared_error"],
        metrics=["mean_squared_error"])

    print(model.summary())
    print("Architecture: U-Net")

    return model

# Callbacks

In [27]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

history_loss = LossHistory() #print(history.losses) to use      

early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, mode='min')    

checkpoint = ModelCheckpoint(filepath = 'checkpoint-{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', save_best_only=False, mode='min', period=1)

#reduce training rate when no improving
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

csv_logger = CSVLogger('training.log')



# Catch SIGINT (same as 'Ctrl+C') signals handler
class SignalStopping(keras.callbacks.Callback):
    """
    Stop training when an interrupt signal (or other) was received
    """

    def __init__(self, sig=signal.SIGINT, doubleSignalExits=False, verbose=0):
        """
        Stop training when an interrupt signal (or other) was received
        
        Keyword Arguments:
            sig {signal} -- The signal to listen to. Defaults to signal.SIGINT. (default: {signal.SIGINT})
            doubleSignalExits {bool} -- Receiving the signal twice exits the python process instead of waiting for this epoch to finish. (default: {False})
            verbose {int} -- Verbosity mode. (default: {0})
        """

        super(SignalStopping, self).__init__()

        self.signal_received = False
        self.verbose = verbose
        self.doubleSignalExits = doubleSignalExits

        def signal_handler(sig, frame):
            if self.signal_received and self.doubleSignalExits:
                if self.verbose > 0:
                    print(
                        ""
                    )  #new line to not print on current status bar. Better solution?
                    print("Received signal to stop " + str(sig) +
                          " twice. Exiting..")
                exit(sig)

            self.signal_received = True
            if self.verbose > 0:
                print(
                    ""
                )  #new line to not print on current status bar. Better solution?
                print("Received signal to stop: " + str(sig))

        signal.signal(signal.SIGINT, signal_handler)
        self.stopped_epoch = 0

    def on_epoch_end(self, epoch, logs={}):
        if self.signal_received:
            self.stopped_epoch = epoch
            self.model.stop_training = True

    def on_train_end(self, logs={}):
        if self.stopped_epoch > 0 and self.verbose > 0:
            print("Epoch %05d: stopping due to signal" % (self.stopped_epoch))


# Training Functions

In [28]:
def training(num_train, num_validation, model_architecture, train_generator, validation_generator, epochs, batch_size, weights_path=None):
    
    #num_classes = train_generator.num_classes
    
    input_tensor = Input(shape=(256, 256, 3)) 
    input_shape=(256, 256, 3)
    nb_classes=1
    
    model = unet(input_tensor, input_shape, nb_classes) 
    
    print(model.summary())
    #print_model(model,"model_plus.png")
    
    history = model.fit_generator(
        train_generator, 
        steps_per_epoch = math.ceil(num_train/batch_size),
        epochs=epochs,
        validation_data = validation_generator,
        validation_steps = math.ceil(num_validation/batch_size),
        callbacks=[SignalStopping, history_loss, early_stop, checkpoint, reduce_lr, csv_logger],
        verbose=1) 
    
    model.save(model, problem_name + '_' + model_architecture + '.h5') 
    
    print_history_accuracy(history) 
    print_history_loss(history) 
    # Final evaluation with test cases
    scores = model.evaluate_generator(validation_generator) 
    print('Scores: ', scores) 
    print("Accuracy: %.2f%%" % (scores[1]*100)) 
    print("Erro modelo: %.2f%%" % (100-scores[1]*100))
    
    return model, history 

def training_folders(model_architecture, trainPath, testPath, target_size, epochs, batch_size, weights_path=None):
    
    # we create two instances with the same arguments
    data_gen_args = dict(validation_split = 0.1,#featurewise_center=True,
                         #featurewise_std_normalization=True,
                         rotation_range=20.,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         zoom_range=[1.5,1.5])
    
    image_datagen = ImageDataGenerator(**data_gen_args)
    mask_datagen = ImageDataGenerator(**data_gen_args)

    # Provide the same seed and keyword arguments to the fit and flow methods
    seed = 1
    #image_datagen.fit(images, augment=True, seed=seed)
    #mask_datagen.fit(masks, augment=True, seed=seed)

    image_generator = image_datagen.flow_from_directory(
        trainPath + 'images',
        class_mode=None,
        seed=seed,
        target_size=target_size,
        color_mode = 'rgb',
        batch_size=batch_size,
        #shuffle = True)
        subset = 'training'
    )
    mask_generator = mask_datagen.flow_from_directory(
        'train/masks',
        class_mode=None,
        seed=seed,
        target_size=target_size,
        color_mode = 'rgb',
        batch_size=batch_size,
        #shuffle = True)
        subset = 'training'
    )
    
    # combine generators into one which yields image and masks
    train_generator = zip(image_generator, mask_generator)
    
    test_image_generator = image_datagen.flow_from_directory(
        'train/images',
        class_mode=None,
        seed=seed,
        target_size=target_size,
        color_mode = 'rgb',
        batch_size=batch_size,
        #shuffle = True)
        subset = 'validation'
    )
    
    test_mask_generator = mask_datagen.flow_from_directory(
        'train/masks',
        class_mode=None,
        seed=seed,
        target_size=target_size,
        color_mode = 'rgb',
        batch_size=batch_size,
        #shuffle = True)
        subset = 'validation'
    )
    
    # combine generators into one which yields image and masks
    test_generator = zip(test_image_generator, test_mask_generator)

    num_train = image_generator.n
    num_validation = test_image_generator.n
    '''  
    train_datagen = ImageDataGenerator(         #https://keras.io/preprocessing/image/
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
    
    test_datagen = ImageDataGenerator(
        rescale=1./255)
    
    train_generator = train_datagen.flow_from_directory(
            trainPath,
            target_size=target_size,
            color_mode = 'rgb',
            batch_size=batch_size,
            class_mode = None,
            shuffle = True)

    validation_generator = test_datagen.flow_from_directory(
            testPath,
            target_size = target_size,
            color_mode = 'rgb',
            class_mode = None,
            batch_size = batch_size,
            shuffle = True)
    ''' 
    model, history = training(num_train, num_validation, model_architecture, train_generator, test_generator, epochs, batch_size, weights_path)
    
    return model, history 

# Training

In [ ]:
if __name__ == '__main__':
                             
    model, history = training_folders(model_architecture, trainPath, testPath, target_size, epochs, batch_size, weights_path)


Found 2340 images belonging to 1 classes.
Found 2340 images belonging to 1 classes.
Found 260 images belonging to 1 classes.
Found 260 images belonging to 1 classes.
Architecture: U-Net
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_116 (Conv2D)             (None, 256, 256, 32) 896         input_10[0][0]                   
__________________________________________________________________________________________________
conv2d_117 (Conv2D)             (None, 256, 256, 32) 9248        conv2d_116[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_29 (MaxP

conv2d_129 (Conv2D)             (None, 64, 64, 128)  147584      conv2d_128[0][0]                 
__________________________________________________________________________________________________
conv2d_transpose_23 (Conv2DTran (None, 128, 128, 64) 32832       conv2d_129[0][0]                 
__________________________________________________________________________________________________
concatenate_23 (Concatenate)    (None, 128, 128, 128 0           conv2d_transpose_23[0][0]        
                                                                 conv2d_119[0][0]                 
__________________________________________________________________________________________________
conv2d_130 (Conv2D)             (None, 128, 128, 64) 73792       concatenate_23[0][0]             
__________________________________________________________________________________________________
conv2d_131 (Conv2D)             (None, 128, 128, 64) 36928       conv2d_130[0][0]                 
__________

# Testing

In [1]:
model_name = 'VGG_16'
testPath = 'test/'

model = load_model('Models/' + model_name + 'model.h5')

test_datagen = ImageDataGenerator(
    rescale=1./255)

validation_generator = test_datagen.flow_from_directory(
        testPath,
        color_mode = 'rgb',
        class_mode = 'categorical',
        target_size=target_size,
        batch_size=batch_size,
        shuffle=False)  # keep data in same order as labels


# Final evaluation with test cases
scores = model.evaluate_generator(validation_generator) 
print('Scores: ', scores) 
print("Accuracy: %.2f%%" % (scores[1]*100)) 
print("Erro modelo: %.2f%%" % (100-scores[1]*100))

scores = model.predict_generator(validation_generator)
j, predicted_classes = np.unravel_index(scores.argmax(axis=1), scores.shape)

for i in range(0, scores.shape[0]):
    if predicted_classes[i] != validation_generator.classes[i]:#print failed images
        plt.imshow(imread(pathtest + validation_generator.filenames[i]))
        plt.show()
        print('true: ', validation_generator.classes[i])
        print('prediction: ',predicted_classes[i])
        print('scores: ', scores[i])
        print('file: ', validation_generator.filenames[i])
    

NameError: name 'ImageDataGenerator' is not defined

# Confusion Matrix + AUC

In [ ]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(validation_generator.classes, predicted_classes))

# AUC for prediction on validation sample
X_val_sample, val_labels = next(validation_generator)
val_pred = model.predict_proba(X_val_sample)
val_pred = np.reshape(val_pred, val_labels.shape)
val_score_auc = roc_auc_score(val_labels, val_pred)
print ("AUC validation score: ",val_score_auc)

# Data Augmentation

In [ ]:
dir_to_save = 'testes'
n_imgs = 10
#select image 
origin = '/media/marcelo/OS/Users/Marcelo Queirós/Documents/MIEI/Semestre 2/Tecnologias e Aplicacoes/deep learning/Projects/pills/dc/' 
image_path = origin + '1.jpg'

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=2,
    height_shift_range=2,
    horizontal_flip=True)

image = np.expand_dims(ndimage.imread(image_path),0) 


# fits the model on batches with real-time data augmentation:
#model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                    #steps_per_epoch=len(x_train) / 32, epochs=epochs)
i = 0
for batch in datagen.flow(image, batch_size=1, 
                          save_to_dir = dir_to_save, save_prefix='dc', save_format='jpg'):
    i += 1
    if i > n_imgs:
        break  # otherwise the generator would loop indefinitely
        
#plots(aug_images, figsize=(20,7), rows=2)